## Load Libraries & Datasets

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from scipy.io import savemat

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from keras.utils.np_utils import to_categorical

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import auc, average_precision_score, precision_recall_curve, roc_curve

In [2]:
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Flatten
from keras import optimizers
from keras import metrics

from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from random import sample

In [3]:
def prepare_dset(dset):
    dset = pd.concat([pd.DataFrame(np.asarray(dset.conc > 0.01, dtype='int')).rename(columns={0:'label'}), 
           dset.iloc[:, 3:]], axis=1)
    return dset

In [4]:
## raw data
pbno3_b1_raw = pd.read_csv("./data_final/pbno3_dset/new/pbno3_batch1.csv")
pbno3_b2_raw = pd.read_csv("./data_final/pbno3_dset/new/pbno3_batch2.csv")
pbno3_b1_raw = prepare_dset(pbno3_b1_raw)
pbno3_b2_raw = prepare_dset(pbno3_b2_raw)

## Power Spectrum Normalized
pbno3_b1_psn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_psn_batch1.csv")
pbno3_b2_psn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_psn_batch2.csv")
pbno3_b1_psn = prepare_dset(pbno3_b1_psn)
pbno3_b2_psn = prepare_dset(pbno3_b2_psn)

## Baseline Corrected
pbno3_b1_bc = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_batch1.csv")
pbno3_b2_bc = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_batch2.csv")
pbno3_b1_bc = prepare_dset(pbno3_b1_bc)
pbno3_b2_bc = prepare_dset(pbno3_b2_bc)

## Define Util Functions

In [5]:
## Define performance measures
def yoden_index(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    j = (tp/(tp+fn)) + (tn/(tn+fp)) - 1
    return j

def pmeasure(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    sensitivity = tp / (tp + fn )
    specificity = tn / (tn + fp)
    f1score = (2 * tp) / (2 * tp + fp + fn)
    return ({'Sensitivity': sensitivity, 'Specificity': specificity, 'F1-Score': f1score})

def Show_Statistics(msg,mean_Stats, sd_Stats, sigfig):
    print(msg.upper())
    print(70*'-')
    print('Accuracy:{} + {}'          .format(round(mean_Stats[0],sigfig), round(sd_Stats[0],sigfig)))
    print('Sensitivity:{} + {} '      .format(round(mean_Stats[1],sigfig), round(sd_Stats[1],sigfig)))
    print('Specificity:{} + {}'       .format(round(mean_Stats[2],sigfig), round(sd_Stats[2],sigfig)))
    print('F1-Score:{} + {}'          .format(round(mean_Stats[3],sigfig), round(sd_Stats[3],sigfig)))
    print('MCC:{} + {}'               .format(round(mean_Stats[4],sigfig), round(sd_Stats[4],sigfig)))
    print('Balance Accuracy:{} + {}'  .format(round(mean_Stats[5],sigfig), round(sd_Stats[5],sigfig)))
    print('Youden-Index:{} + {}'      .format(round(mean_Stats[6],sigfig), round(sd_Stats[6],sigfig)))
    print('AUC:{} + {}'               .format(round(mean_Stats[7],sigfig), round(sd_Stats[7],sigfig)))
    print('AUPR:{} + {}'              .format(round(mean_Stats[8],sigfig), round(sd_Stats[8],sigfig)))  
    print(70*'-')

def Calculate_Stats(y_actual,y_pred, y_score):
    acc = accuracy_score(y_actual, y_pred)
    sen = pmeasure(y_actual, y_pred)['Sensitivity']
    spe = pmeasure(y_actual, y_pred)['Specificity']
    f1 = pmeasure(y_actual, y_pred)['F1-Score']
    mcc = matthews_corrcoef(y_actual, y_pred)
    bacc = balanced_accuracy_score(y_actual, y_pred)
    yi = yoden_index(y_actual, y_pred)
    #auc = roc_auc_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  
    pre, rec, _ = precision_recall_curve(y_actual, y_score, pos_label=1)
    fpr, tpr, _ = roc_curve(y_actual, y_score, pos_label=1)
    auroc = auc(fpr, tpr)
    aupr = auc(rec, pre)
    
    return acc, sen, spe, f1, mcc, bacc, yi, auroc, aupr  

def label_by_th(y_pred, threshold=0.5):
    y_pred_copy = y_pred.copy()
    y_pred_copy[y_pred>= threshold] = 1 
    y_pred_copy[y_pred<threshold] = 0 
    return y_pred_copy

def cutoff_youdens_j(fpr,tpr,thresholds):
    j_scores = tpr-fpr
    j_ordered = sorted(zip(j_scores,thresholds))
    return j_ordered[-1][1]

In [6]:
def get_perf_summary(dataset, model, type):
    ## Prepare dataset
    dataset.loc[dataset.label>=1,"label"]=1
    
    ## Datasets 
    [X, y_true] = np.array(dataset.iloc[:,1:]), np.array(dataset.iloc[:,0])
    y_true = to_categorical(y_true)
    if type=="regression":
        y_score = model.predict(X, batch_size=1800, verbose=0)    
    else:
        y_score = model.predict(X, batch_size=1800, verbose=0)[:,1]
    #y_score = model.predict(X,batch_size=1800, verbose=0)
    #y_pred = to_categorical(get_label(y_score))      

    # Optimal Threshold
    fpr, tpr, thresholds_AUC = roc_curve(y_true.argmax(axis=1), y_score)
    precision, recall, thresholds_AUPR = precision_recall_curve(y_true.argmax(axis=1),y_score)

    ## Optimal Threshold metrics
    distance = (1-fpr)**2+(1-tpr)**2
    EERs = (1-recall)/(1-precision)
    positive = sum(y_true.argmax(axis=1))
    negative = y_true.shape[0]-positive
    ratio = negative/positive
    opt_t_AUC = thresholds_AUC[np.argmin(distance)]
    opt_t_AUPR = thresholds_AUPR[np.argmin(np.abs(EERs-ratio))]
    opt_yodens_j = cutoff_youdens_j(fpr, tpr, thresholds_AUC)        
    y_pred_th = label_by_th(y_score, opt_yodens_j)
    y_pred = to_categorical(y_pred_th)

    print(confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1), labels=[0,1]).ravel())    

    ## Measures
    t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(y_true, y_pred, y_score)

    print("ACC: {}, Sen: {}, Spe: {}, F1: {}, MCC: {}, BACC: {}, YI: {}, AUC: {}, AUPR: {}".format(t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr))

## Define Classifiers

In [7]:
def get_clf_res_ML(dev_dset, ind_test_dset, method):
    num_Trials = 10    

    # import warnings filter
    from warnings import simplefilter
    # ignore all future warnings
    simplefilter(action='ignore', category=FutureWarning)
    [DataX, LabelY] = np.array(dev_dset.iloc[:,1:]), np.array(dev_dset.iloc[:,0])    

    Stats =[]
    Stats_Ind =[]
    for j in range(0, num_Trials):        
        X_train, X_val, y_train, y_val =\
        train_test_split(DataX, LabelY, test_size=0.2, stratify = LabelY)
        X_val, X_test, y_val, y_test =\
        train_test_split(X_val, y_val, test_size=0.5, stratify = y_val)

        if method == "LR":
            clf = LogisticRegression(random_state=0).fit(X_train, y_train)

        elif method == "LinSVM":
            clf = SVC(kernel = 'linear', probability=True).fit(X_train, y_train)

        elif method == "NB":
            clf = GaussianNB().fit(X_train, y_train)

        elif method == "DT":
            clf = DecisionTreeClassifier().fit(X_train, y_train)

        elif method == "MLP":
            clf = MLPClassifier().fit(X_train, y_train)

        elif method == "RF":
            clf = RandomForestClassifier().fit(X_train, y_train)

        elif method == "RBFSVM":
            clf = SVC(kernel = 'rbf', probability=True).fit(X_train, y_train)

        else:
            print("Please check avilable models")
            break;

        y_train_pred = clf.predict(X_train)
        y_train_score = clf.predict_proba(X_train)     
        y_train_pred = to_categorical(y_train_score.argmax(axis=1))
        y_train_score = y_train_score[:,1]

        y_val_pred = clf.predict(X_val)
        y_val_score = clf.predict_proba(X_val)     
        y_val_pred = to_categorical(y_val_score.argmax(axis=1))
        y_val_score = y_val_score[:,1]

        y_test_pred = clf.predict(X_test)
        y_test_score = clf.predict_proba(X_test) 
        y_test_pred = to_categorical(y_test_score.argmax(axis=1))
        y_test_score = y_test_score[:,1] 

        print(confusion_matrix(y_test, y_test_pred.argmax(axis=1), labels=[0,1]).ravel())

        y_train_pred = y_train_pred.argmax(axis=1)
        y_val_pred = y_val_pred.argmax(axis=1)
        y_test_pred = y_test_pred.argmax(axis=1)

        ## Training Measures
        tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr = Calculate_Stats(y_train,y_train_pred, y_train_score);

        ## Validation Measures
        v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, v_auc, v_aupr = Calculate_Stats(y_val,y_val_pred, y_val_score);

        ## Test Measures
        t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(y_test,y_test_pred, y_test_score);

        Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr,
                      v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, v_auc, v_aupr,
                      t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr])  

        print(' \nTraining/ Validation / Test BACC :', tr_bacc,'/',v_bacc,'/',t_bacc,
              ' \nTraining/ Validation / Test Youden-index:', tr_yi,'/',v_yi,'/',t_yi,
              ' \nTraining/ Validation / Test MCC:', tr_mcc,'/',v_mcc,'/',t_mcc,
              ' \nTraining/ Validation / Test AUC:', tr_auc,'/',v_auc,'/',t_auc,
              ' \nTraining/ Validation / Test AUPR:', tr_aupr,'/',v_aupr,'/',t_aupr)    

        ## Independent Test
        [Xt1, Yt1] = np.array(ind_test_dset.iloc[:,1:]), np.array(ind_test_dset.iloc[:,0])

        Yt1_pred = clf.predict(Xt1)
        Yt1_score = clf.predict_proba(Xt1)

        Stats_Ind.append(Calculate_Stats(Yt1, Yt1_pred, Yt1_score[:,1]))

        print(Stats_Ind)
    
    return Stats, Stats_Ind

In [8]:
def format_perf_mat(perf_list, model_type):
    perf_tbl = pd.DataFrame(perf_list)
    perf_tbl = pd.concat([perf_tbl, pd.DataFrame(np.repeat(model_type, len(perf_list)))], axis=1)
    perf_tbl.columns = ['Accuracy', "Sensitivity", "Specificity", "F1", "MCC", "BACC", 
                        "Yoden Index", "AUROC", "AUPR", "Model"]    
    return perf_tbl

In [9]:
def get_all_clf_res(devset, indset, species, prep_method):
    NB_cv, NB_ind = get_clf_res_ML(devset, indset, method='NB')
    DT_cv, DT_ind = get_clf_res_ML(devset, indset, method='DT')
    LR_cv, LR_ind = get_clf_res_ML(devset, indset, method='LR')
    RF_cv, RF_ind = get_clf_res_ML(devset, indset, method='RF')
    LinSVM_cv, LinSVM_ind = get_clf_res_ML(devset, indset, method='LinSVM')
    RBFSVM_cv, RBFSVM_ind = get_clf_res_ML(devset, indset, method='RBFSVM')
    MLP_cv, MLP_ind = get_clf_res_ML(devset, indset, method='MLP')
    
    fold_ind_perf = pd.concat([format_perf_mat(NB_ind, 'NB'), 
                               format_perf_mat(DT_ind, 'DT'), 
                               format_perf_mat(LR_ind, 'LR'),
                               format_perf_mat(RF_ind, 'RF'), 
                               format_perf_mat(LinSVM_ind, 'LinSVM'),
                               format_perf_mat(RBFSVM_ind, 'RBFSVM'),
                               format_perf_mat(MLP_ind, 'MLP')], axis=0)

    return fold_ind_perf

## Train and Evaluate Performance of Models

In [ ]:
perf_raw_pbno3_1 = get_all_clf_res(pbno3_b1_raw, pbno3_b2_raw, 'pbno3_b1b2', 'raw')
perf_raw_pbno3_2 = get_all_clf_res(pbno3_b2_raw, pbno3_b1_raw, 'pbno3_b2b1', 'raw')

perf_psn_pbno3_1 = get_all_clf_res(pbno3_b1_psn, pbno3_b2_psn, 'pbno3_b1b2', 'psn')
perf_psn_pbno3_2 = get_all_clf_res(pbno3_b2_psn, pbno3_b1_psn, 'pbno3_b2b1', 'psn')

perf_bn_pbno3_1 = get_all_clf_res(pbno3_b1_bc, pbno3_b2_bc, 'pbno3_b1b2', 'bc')
perf_bn_pbno3_2 = get_all_clf_res(pbno3_b2_bc, pbno3_b1_bc, 'pbno3_b2b1', 'bc')

[ 49   1   0 150]
 
Training/ Validation / Test BACC : 0.9862500000000001 / 1.0 / 0.99  
Training/ Validation / Test Youden-index: 0.9725000000000001 / 1.0 / 0.98  
Training/ Validation / Test MCC: 0.9816651116404495 / 1.0 / 0.986666070043373  
Training/ Validation / Test AUC: 0.98625 / 1.0 / 0.99  
Training/ Validation / Test AUPR: 0.9954582989265071 / 1.0 / 0.9966887417218543
[(0.5455, 0.7273333333333334, 0.0, 0.7059204141054675, -0.2927291265048543, 0.3636666666666667, -0.2726666666666666, 0.3636666666666667, 0.808968918077167)]
[ 48   2   0 150]
 
Training/ Validation / Test BACC : 0.9875 / 1.0 / 0.98  
Training/ Validation / Test Youden-index: 0.9750000000000001 / 1.0 / 0.96  
Training/ Validation / Test MCC: 0.9833321660356334 / 1.0 / 0.9733285267845753  
Training/ Validation / Test AUC: 0.9875 / 1.0 / 0.98  
Training/ Validation / Test AUPR: 0.9958677685950413 / 1.0 / 0.993421052631579
[(0.5455, 0.7273333333333334, 0.0, 0.7059204141054675, -0.2927291265048543, 0.3636666666666667

[ 49   1   0 150]
 
Training/ Validation / Test BACC : 0.98 / 0.98 / 0.99  
Training/ Validation / Test Youden-index: 0.96 / 0.96 / 0.98  
Training/ Validation / Test MCC: 0.9733285267845753 / 0.9733285267845753 / 0.986666070043373  
Training/ Validation / Test AUC: 0.9825 / 0.98 / 0.99  
Training/ Validation / Test AUPR: 0.9942339373970346 / 0.993421052631579 / 0.9966887417218543
[(0.5455, 0.7273333333333334, 0.0, 0.7059204141054675, -0.2927291265048543, 0.3636666666666667, -0.2726666666666666, 0.3636666666666667, 0.808968918077167), (0.545, 0.7266666666666667, 0.0, 0.7055016181229773, -0.2931789187624318, 0.36333333333333334, -0.2733333333333333, 0.36333333333333334, 0.8087747626878982), (0.544, 0.7253333333333334, 0.0, 0.7046632124352331, -0.29407813237795344, 0.3626666666666667, -0.2746666666666666, 0.36233333333333334, 0.8082263272993567), (0.5445, 0.726, 0.0, 0.7050825509873746, -0.2936285868417763, 0.363, -0.274, 0.3626666666666667, 0.808407774965137), (0.5465, 0.728666666666666

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.383, 0.5106666666666667, 0.0, 0.5538684020245842, -0.43961302337535646, 0.25533333333333336, -0.4893333333333333, 0.25533333333333336, 0.7413609794628752), (0.576, 0.768, 0.0, 0.7309644670050761, -0.26498663678673456, 0.384, -0.23199999999999998, 0.384, 0.8196682808716707), (0.401, 0.5346666666666666, 0.0, 0.5724482512491078, -0.4227285615663471, 0.2673333333333333, -0.4653333333333334, 0.2673333333333333, 0.749821044546851), (0.567, 0.756, 0.0, 0.7236758136566688, -0.2732460478996511, 0.378, -0.244, 0.378, 0.816501223990208), (0.477, 0.636, 0.0, 0.6459038591740014, -0.3537964661741324, 0.318, -0.364, 0.318, 0.782560522696011), (0.3645, 0.486, 0.0, 0.5342616342982778, -0.45728885847062467, 0.243, -0.51

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.5885, 0.4573333333333333, 0.982, 0.6250569476082004, 0.39950951671609486, 0.7196666666666667, 0.43933333333333335, 0.7389533333333334, 0.9128651681178219), (0.505, 0.34, 1.0, 0.5074626865671642, 0.3377779740177616, 0.67, 0.3400000000000001, 0.673916, 0.8950540505852567), (0.5325, 0.37666666666666665, 1.0, 0.5472154963680388, 0.3622744001428179, 0.6883333333333334, 0.3766666666666667, 0.7289760000000001, 0.912321874727281), (0.521, 0.36133333333333334, 1.0, 0.5308521057786484, 0.35201425751563586, 0.6806666666666666, 0.3613333333333333, 0.7337093333333333, 0.9142075367573501), (0.519, 0.3586666666666667, 1.0, 0.5279685966633955, 0.3502328064243132, 0.6793333333333333, 0.3586666666666667, 0.7280973333333

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.45, 0.6, 0.0, 0.6206896551724138, -0.3779644730092272, 0.3, -0.4, 0.22421866666666668, 0.7162552881090564), (0.466, 0.6213333333333333, 0.0, 0.635743519781719, -0.36361520168658945, 0.31066666666666665, -0.3786666666666667, 0.21956533333333333, 0.7158202255629782), (0.4745, 0.6326666666666667, 0.0, 0.643608002712784, -0.3560253969533949, 0.31633333333333336, -0.3673333333333333, 0.23431000000000002, 0.7206869267214899), (0.445, 0.5933333333333334, 0.0, 0.615916955017301, -0.3824695721506524, 0.2966666666666667, -0.4066666666666666, 0.2371346666666667, 0.7239599246005912), (0.4645, 0.6193333333333333, 0.0, 0.6343461932400136, -0.3649566759503614, 0.30966666666666665, -0.3806666666666667, 0.2347913333333

...

[(0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.30134866666666665, 0.735526597227498), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.30293666666666663, 0.7361450470489113)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 0.9999999999999999 / 0.9999999999999999


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[(0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.30134866666666665, 0.735526597227498), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.30293666666666663, 0.7361450470489113), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.293442, 0.7320515409516117)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 0.9999999999999999


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[(0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.30134866666666665, 0.735526597227498), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.30293666666666663, 0.7361450470489113), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.293442, 0.7320515409516117), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.30252866666666667, 0.7361619774707534)]
